## __PROJET 5 - IMDB Movie Prediction__ - _*Ludovic & Yasemin*_

####  __SECONDE PARTIE__ : MACHINE LEARNING

In [8]:
#Import des librairies nécessaires
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [9]:
#Chargement du dataset nettoyé
pd.set_option("display.max_columns", None)

df = pd.read_csv("./Datas/5000_movies_bis_clean.csv")
df.head(3)

,Unnamed: 0,duration,director_fb_likes,actor_1_fb_likes,gross,num_voted_users,facenumber_in_poster,budget,title_year,aspect_ratio,movie_fb_likes,country_UK,country_USA,other_actors_fb_likes,critic_reviews_ratio,imdb_classification
0,0,178.0,0.0,1000.0,760505847.0,886204,0.0,237000000.0,2009.0,1.78,33000,False,True,1791.0,0.236739,2
1,1,169.0,563.0,40000.0,309404152.0,471220,0.0,300000000.0,2007.0,2.35,0,False,True,6000.0,0.243942,2
2,2,148.0,0.0,11000.0,200074175.0,275868,1.0,245000000.0,2015.0,2.35,85000,True,False,554.0,0.605634,2


In [10]:
#Rappel des infos actualisées de notre dataset nettoyé
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4703 entries, 0 to 4702
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             4703 non-null   int64  
 1   duration               4703 non-null   float64
 2   director_fb_likes      4703 non-null   float64
 3   actor_1_fb_likes       4703 non-null   float64
 4   gross                  4703 non-null   float64
 5   num_voted_users        4703 non-null   int64  
 6   facenumber_in_poster   4703 non-null   float64
 7   budget                 4703 non-null   float64
 8   title_year             4703 non-null   float64
 9   aspect_ratio           4703 non-null   float64
 10  movie_fb_likes         4703 non-null   int64  
 11  country_UK             4703 non-null   bool   
 12  country_USA            4703 non-null   bool   
 13  other_actors_fb_likes  4703 non-null   float64
 14  critic_reviews_ratio   4703 non-null   float64
 15  imdb

### Modèle de ML : __Random Forest__

In [11]:
#On prépare les données pour l'entrainement
#On supprime la variable cible, ainsi que "Unnamed: 0" qui n'a pas de valeur prédictive
X = df.drop(["Unnamed: 0", "imdb_classification"], axis=1)
y = df["imdb_classification"]

#On divise le dataset en ensemble entrainement/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#On va maintenant pouvoir entraîner le modèle de classification RF 
#et faire des prédictions sur l'ensemble du test
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#On évalue ce que retourne le modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#Rapport détaillé de classification
print(classification_report(y_test, y_pred))


Accuracy: 0.7290116896918172
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.64      0.46      0.53       272
           2       0.75      0.91      0.82       597
           3       0.89      0.39      0.54        41

    accuracy                           0.73       941
   macro avg       0.57      0.44      0.47       941
weighted avg       0.70      0.73      0.70       941



#### *Commentaires* :
- Ici, le modèle de classification RF prédit correctement la catégorie dans environ 73% du temps sur l'ensemble du dataset
- Cependant, les classes ne sont pas distribuées de manière égale. En effet, les classes "0" et "3" sont mal représentées selon le rapport de classification. La classe 2 est quant à elle bien performée par le modèle, peut être même un peu trop : Faux positifs??

On va tenter d'améliorer le modèle afin d'obtenir des résultats davantage satisfaisants